In [181]:
import pandas as pd
import json
import datetime

In [182]:
df_redefine = pd.read_csv('./raw_data/ariel_project_data_1Kdays.csv', index_col=0)

In [183]:
df_redefine['other_contracts'] = df_redefine['other_contracts'].apply(lambda x: x.replace("'",'"'))
df_redefine['other_contracts'] = df_redefine['other_contracts'].apply(lambda x: json.loads(x))


In [184]:
def parse_contracts(obj):
    x=[]
    for address in obj:
        y = address['smartContract']['address']['address']
        x.append(y)
    return x
  
df_redefine['other_created_address'] = df_redefine['other_contracts'].apply(lambda x: parse_contracts(x))


In [185]:
def parse_date(obj):
    addresses=[]
    dates =[]
    for i in obj:
        address = i['smartContract']['address']['address']
        date = i['date']['date']
        addresses.append(address)
        dates.append(date)
    return addresses, dates

my_tuple = df_redefine['other_contracts'].apply(lambda x: parse_date(x)).tolist()
df_contract_creation_date = pd.DataFrame(my_tuple)
# Assuming the DataFrame is named 'df'
pairs = []

for index, row in df_contract_creation_date.iterrows():
    addresses = row[0]
    dates = row[1]

    # Zip addresses and dates to create pairs
    combined = zip(addresses, dates)
    pairs.extend(combined)

# Create a new DataFrame using the pairs and the specified column names
df_contract_creation_date = pd.DataFrame(pairs, columns=['address', 'date'])


In [186]:
df_contract_creation_date

,address,date
0,0xb83e7f1b67d9e57faaa82452ddea2ec9db54f2e8,2022-09-05
1,0x4c37c126eac3baaa98e38eae05215c522e6ca817,2021-09-10
2,0x185486869e615f5a5644991a491212582caa08fb,2021-02-28
3,0x4a5020dc88541c4e0ed19fd7e1da2f6722012c56,2021-02-28
4,0x40730f34668afcb3884f050cbc3d376a444bbe44,2021-01-25
...,...,...
15408,0x60c6b5dc066e33801f2d9f2830595490a3086b4e,2021-11-05
15409,0x3c40601f73fbf50b81a72edbf2786f14ebb7371b,2021-10-19
15410,0xbef1b61dc62b82659f102a332e1276e67371ae6e,2021-10-19
15411,0xf8e9f10c22840b613cda05a0c5fdb59a4d6cd7ef,2021-10-15


In [187]:
df_redefine.columns

Index(['token_address', 'project_name', 'code_repo', 'token_creator',
       'total_tvl', 'other_contracts', 'token_creation_date',
       'other_created_address'],
      dtype='object')

In [188]:
df_redefine_exploded = df_redefine.loc[:,['token_creator','other_created_address','project_name','token_creation_date']]

In [189]:
df_redefine_exploded = df_redefine_exploded.explode('other_created_address')

In [190]:
df_redefine_exploded

,token_creator,other_created_address,project_name,token_creation_date
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0xb83e7f1b67d9e57faaa82452ddea2ec9db54f2e8,Matic,2019-04-20
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x4c37c126eac3baaa98e38eae05215c522e6ca817,Matic,2019-04-20
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x185486869e615f5a5644991a491212582caa08fb,Matic,2019-04-20
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x4a5020dc88541c4e0ed19fd7e1da2f6722012c56,Matic,2019-04-20
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x40730f34668afcb3884f050cbc3d376a444bbe44,Matic,2019-04-20
...,...,...,...,...
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0x60c6b5dc066e33801f2d9f2830595490a3086b4e,Dogs Of Elon,2021-10-15
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0x3c40601f73fbf50b81a72edbf2786f14ebb7371b,Dogs Of Elon,2021-10-15
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0xbef1b61dc62b82659f102a332e1276e67371ae6e,Dogs Of Elon,2021-10-15
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0xf8e9f10c22840b613cda05a0c5fdb59a4d6cd7ef,Dogs Of Elon,2021-10-15


In [191]:
# df_merge = pd.merge(df_redefine, df_etherscan, left_on="other_created_address",right_on="index", how="left")
df_redefine_with_date = pd.merge(df_redefine_exploded, df_contract_creation_date, left_on='other_created_address', right_on='address', how='left')
df_redefine_with_date.drop('address', axis=1,inplace=True)


In [195]:
df_redefine_with_date = df_redefine_with_date.loc[df_redefine_with_date['date'].notna()]

In [198]:
def to_datetime(obj):
    format_string = '%Y-%m-%d'
    datetime_obj = datetime.datetime.strptime(obj, format_string)
    return datetime_obj



df_redefine_with_date['date'] = df_redefine_with_date['date'].apply(lambda x: to_datetime(x))
df_redefine_with_date['token_creation_date'] = df_redefine_with_date['token_creation_date'].apply(lambda x: to_datetime(x))



/var/folders/00/_mvycxm948781kjwpdqn_f1w0000gn/T/ipykernel_61524/3804249097.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_redefine_with_date['date'] = df_redefine_with_date['date'].apply(lambda x: to_datetime(x))
/var/folders/00/_mvycxm948781kjwpdqn_f1w0000gn/T/ipykernel_61524/3804249097.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_redefine_with_date['token_creation_date'] = df_redefine_with_date['token_creation_date'].apply(lambda x: to_datetime(x))


In [199]:
df_redefine_with_date['date_creation_diff']=df_redefine_with_date.date - df_redefine_with_date.token_creation_date

/var/folders/00/_mvycxm948781kjwpdqn_f1w0000gn/T/ipykernel_61524/3883998346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_redefine_with_date['date_creation_diff']=df_redefine_with_date.date - df_redefine_with_date.token_creation_date


In [202]:
df_redefine_with_date['date_creation_diff']=df_redefine_with_date['date_creation_diff'].apply(lambda x: int(x.days))

/var/folders/00/_mvycxm948781kjwpdqn_f1w0000gn/T/ipykernel_61524/3230992190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_redefine_with_date['date_creation_diff']=df_redefine_with_date['date_creation_diff'].apply(lambda x: int(x.days))


In [204]:
df_redefine_with_date.to_csv('./raw_data/expanded_redefine_data_with_date.csv')